In [0]:
###############################################hkhkkkhh_hrhrrrhh########################################################

#Important Concepts

###Eager Execution:  
TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, without building graphs: operations return concrete values instead of constructing a computational graph to run later. This makes it easy to get started with TensorFlow and debug models

###Teacher Forcing:   
During training time model receives ground truth output y(t) as input at time t+1. Maximum likelihood specifies that during training, rather than feeding the model’s own output back to itself, target values should specify what the correct output should be. 
####Train time: 
We feed the correct output y(t) (from teacher) drawn from the training set as input. 
####Test time: 
True output is not known. We approximate the correct output y(t) with the model’s output o(t) and feed the output back to the model.

#Main Code

##Imports

In [0]:
!pip install tf-nightly-2.0-preview
import tensorflow as tf

# We'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

    100% |████████████████████████████████| 77.9MB 583kB/s 
    100% |████████████████████████████████| 358kB 21.9MB/s 
    100% |████████████████████████████████| 61kB 23.7MB/s 
    100% |████████████████████████████████| 3.2MB 12.0MB/s 


In [0]:
import numpy as np
import cv2

In [0]:
import os

In [0]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
!pip install scikit-video
import numpy as np
import cv2
import dlib
import math
import sys
import pickle
import argparse
import os
import skvideo.io

##Video Processing

###Download dlib files and grid tar files

In [0]:
#!wget http://spandh.dcs.shef.ac.uk/gridcorpus/examples/id2_vcd_swwp2s.mpg
#!wget http://spandh.dcs.shef.ac.uk/gridcorpus/examples/id23_vcd_priazn.mpg
#!wget http://spandh.dcs.shef.ac.uk/gridcorpus/examples/swwp2s.align
#!wget http://spandh.dcs.shef.ac.uk/gridcorpus/examples/priazn.align

In [0]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

In [0]:
!wget http://spandh.dcs.shef.ac.uk/gridcorpus/s1/video/s1.mpg_6000.part1.tar
!tar -xvf s1.mpg_6000.part1.tar && rm -r s1.mpg_6000.part1.tar

### Define dlib paths and set crop parameters

In [0]:
#videos = ['id23_vcd_priazn.mpg', 'id2_vcd_swwp2s.mpg']
predictor_path = 'shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [0]:
width_crop_max = 100
height_crop_max = 50

### Lip Extract

In [0]:
video_dir='mpg_6000/'
videos = [video_dir + i for i in os.listdir(video_dir)]
videos=sorted(videos)

In [0]:
def make_mouth_crops(video):
  global width_crop_max,height_crop_max

  inputparameters = {}
  outputparameters = {}
  reader = skvideo.io.FFmpegReader(video, inputdict=inputparameters, outputdict=outputparameters)
  video_shape = reader.getShape()
  (num_frames, h, w, c) = video_shape

  max_counter = 150
  total_num_frames = int(video_shape[0])
  num_frames = min(total_num_frames,max_counter)
  counter = 0

  out_frame=[]



  # Loop over all frames.
  for frame in reader.nextFrame():
    #print('frame_shape:', frame.shape)
    # Process the video and extract the frames up to a certain number and then stop processing.
    if counter > num_frames:
      break
    # Detection of the frame
    detections = detector(frame, 1)
    # 20 mark for mouth
    marks = np.zeros((2, 20))
    # All unnormalized face features.

    Features_Abnormal = np.zeros((190, 1))

    # If the face is detected.
    #print(len(detections))

    if len(detections) > 0:
      # Shape of the face.
      shape = predictor(frame, detections[0]) #single face considered 
      co = 0
      # Specific for the mouth.
      for ii in range(48, 68):
        """
        This for loop is going over all mouth-related features.
        X and Y coordinates are extracted and stored separately.
        """
        X = shape.part(ii)
        A = (X.x, X.y)
        marks[0, co] = X.x
        marks[1, co] = X.y
        co += 1

      # Get the extreme points(top-left & bottom-right)
      X_left, Y_left, X_right, Y_right = [int(np.amin(marks, axis=1)[0]), int(np.amin(marks, axis=1)[1]),
                                          int(np.amax(marks, axis=1)[0]),
                                          int(np.amax(marks, axis=1)[1])]
      # Find the center of the mouth.
      X_center = (X_left + X_right) / 2.0
      Y_center = (Y_left + Y_right) / 2.0

      # Make a boarder for cropping.
      border = 15
      X_left_new = X_left - border
      Y_left_new = Y_left - border
      X_right_new = X_right + border
      Y_right_new = Y_right + border

      # Width and height for cropping(before and after considering the border).
      width_new = X_right_new - X_left_new
      height_new = Y_right_new - Y_left_new
      width_current = X_right - X_left
      height_current = Y_right - Y_left


      # Determine the cropping rectangle dimensions(the main purpose is to have a fixed area).
      if width_crop_max == 0 and height_crop_max == 0:
        width_crop_max = width_new
        height_crop_max = height_new
      else:
        width_crop_max += 1.5 * np.maximum(width_current - width_crop_max, 0)
        height_crop_max += 1.5 * np.maximum(height_current - height_crop_max, 0)

      # # # Uncomment if the lip area is desired to be rectangular # # # #
      #########################################################
      # Find the cropping points(top-left and bottom-right).
      X_left_crop = int(X_center - width_crop_max / 2.0)
      X_right_crop = int(X_center + width_crop_max / 2.0)
      Y_left_crop = int(Y_center - height_crop_max / 2.0)
      Y_right_crop = int(Y_center + height_crop_max / 2.0)



      if X_left_crop >= 0 and Y_left_crop >= 0 and X_right_crop < w and Y_right_crop < h:
        mouth = frame[Y_left_crop:Y_right_crop, X_left_crop:X_right_crop, :]
        #cv2.imwrite(mouth_destination_path + '/' + 'frame' + '_' + str(counter) + '.png', mouth)
        out_frame.append(mouth.T)
        counter += 1
      else:
        cv2.putText(frame, 'NA. ', (30, 30), font, 1, (0, 255, 255), 2) #Check in LRS

    else:
      cv2.putText(frame, 'NA. ', (30, 30), font, 1, (0, 0, 255), 2)#Check in LRS
    
  out_frame=np.asarray(out_frame).reshape(-1,100,50,3)
    
  return out_frame

###Process frames

In [0]:
X_train=[]
for i,v in enumerate(videos):
  if i>9:
    break
  out_frame=make_mouth_crops(v)
  X_train.append(out_frame)
  print("Video ",i," : ",out_frame.shape)

In [0]:
X_train = np.asarray(X_train)
X_train = X_train.astype('float32')
X_train /= 255

##Text Processing

### Get transcript tars

In [0]:
!wget http://spandh.dcs.shef.ac.uk/gridcorpus/s1/align/s1.tar
!tar -xvf s1.tar && rm -r s1.tar

In [0]:
#files=['swwp2s.align','priazn.align']
def prep_text(files):
  sent=[]
  for file in files:
    f = open(file, "r")
    dummy=[line.split()[-1] for line in f] #-1 coz that's the word we want, 0 and 1 have timestamps
    dummy[0]='<start>'
    dummy[-1]='<end>'
    sent.append(dummy)
  return sent

In [0]:
text_dir='align/'
files = [text_dir + i for i in os.listdir(text_dir)]
files=sorted(files)

### Prepare transcripts

In [0]:
sent = prep_text(files)
#sent

[['<start>', 'set', 'white', 'with', 'p', 'two', 'soon', '<end>'],
 ['<start>', 'place', 'red', 'in', 'a', 'zero', 'now', '<end>']]

### Tokenize

In [0]:
#CHANGE: Have used keras instead of tf.keras
tokenizer = Tokenizer(oov_token="<unk>", filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(sent)
tokenizer.word_index['<pad>'] = 0
#tokenizer.word_index

In [0]:
sseq = tokenizer.texts_to_sequences(sent)
#sseq

[[2, 4, 5, 6, 7, 8, 9, 3], [2, 10, 11, 12, 13, 14, 15, 3]]

In [0]:
decoder_input_data=pad_sequences(sseq, padding='post')

In [0]:
decoder_input_data=decoder_input_data[:10]

##Making the Dataset

In [0]:
BATCH_SIZE = 2 #2 videos
BUFFER_SIZE = len(X_train)
embedding_dim = 8
units = 256
vocab_size = len(tokenizer.word_index)
N_BATCH = BUFFER_SIZE//BATCH_SIZE

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, decoder_input_data)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

##Model 

###GRUs/LSTMs/BahdanauAttention

In [0]:
#NOTE: GRU outputs state = [h,c] But, LSTM doesn't give this list, instead separately h,c
def gru(units):
  # If you have a GPU, we recommend using the CuDNNGRU layer (it provides a 
  # significant speedup).
  if tf.test.is_gpu_available():
    return tf.keras.layers.CuDNNGRU(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform')
  else:
    return tf.keras.layers.GRU(units, 
                               return_sequences=True, 
                               return_state=True, 
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform')
  
def lstm(units):
  # If you have a GPU, we recommend using the CuDNNLSTM layer (it provides a 
  # significant speedup).
  if tf.test.is_gpu_available():
    return tf.keras.layers.CuDNNLSTM(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform')
  else:
    return tf.keras.layers.LSTM(units, 
                               return_sequences=True, 
                               return_state=True, 
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform')

In [0]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)
  
  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)
    
    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)
    
    # score shape == (batch_size, 64, hidden_size)
    score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
    
    # attention_weights shape == (batch_size, 64, 1)
    # we get 1 at the last axis because we are applying score to self.V
    attention_weights = tf.nn.softmax(self.V(score), axis=1)
    
    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)
    
    return context_vector, attention_weights

###Encoder-Decoder Architecture

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        
        #NOTE: Define as many times as you want to use. Thus, there will be REDUNDANCY
        self.zero1 = tf.keras.layers.ZeroPadding3D(padding=(1,2,2))
        self.conv1 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3,5,5), strides=(1,2,2), kernel_initializer='he_normal')
        self.bn1 = tf.keras.layers.BatchNormalization()
        #activation defined later
        self.spd1 = tf.keras.layers.SpatialDropout3D(0.5)
        self.maxp1 = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2))
        #activation defined later
        self.zero2 = tf.keras.layers.ZeroPadding3D(padding=(1,2,2))
        self.conv2 = tf.keras.layers.Conv3D(filters=64, kernel_size=(3,5,5), strides=(1,1,1), kernel_initializer='he_normal')
        self.bn2 = tf.keras.layers.BatchNormalization()
        #activation defined later
        self.spd2 = tf.keras.layers.SpatialDropout3D(0.5)
        self.maxp2 = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2))
        #activation defined later
        self.zero3 = tf.keras.layers.ZeroPadding3D(padding=(1,1,1))
        self.conv3 = tf.keras.layers.Conv3D(filters=96, kernel_size=(3,3,3), strides=(1,1,1), kernel_initializer='he_normal')
        self.bn3 = tf.keras.layers.BatchNormalization()
        #activation defined later
        self.spd3 = tf.keras.layers.SpatialDropout3D(0.5)
        self.maxp3 = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2))
        
        self.td=tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())
        self.gru=gru(units=256)
        
        
    def call(self, x, hidden):
        zero1 = self.zero1(x)
        conv1 = self.conv1(zero1)
        batc1 = self.bn1(conv1)
        actv1 = tf.nn.relu(batc1)
        drop1 = self.spd1(actv1)
        maxp1 = self.maxp1(drop1)

        zero2 = self.zero2(maxp1)
        conv2 = self.conv2(zero2)
        batc2 = self.bn2(conv2)
        actv2 = tf.nn.relu(batc2)
        drop2 = self.spd2(actv2)
        maxp2 = self.maxp2(drop2)

        zero3 = self.zero3(maxp2)
        conv3 = self.conv3(zero3)
        batc3 = self.bn3(conv3)
        actv3 = tf.nn.relu(batc3)
        drop3 = self.spd3(actv3)
        maxp3 = self.maxp3(drop3)

        resh1 = self.td(maxp3)
        output, state=self.gru(resh1,initial_state = hidden)
        
        #output, state_h, state_c=self.lstm(resh1,initial_state = hidden)
        #state = [state_h, state_c]
        
        return output, state
      
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))
    

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(Decoder, self).__init__()
    self.units = units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = gru(self.units)
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.attention = BahdanauAttention(self.units)
    
    #features = enc_output
  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)
    
    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)
    
    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    
    # passing the concatenated vector to the GRU
    output, state = self.gru(x)
    #output, state_h, state_c = self.lstm(x)
    #state = [state_h, state_c]
    # shape == (batch_size, max_length, hidden_size)
    x = self.fc(output)
    
    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))
        

In [0]:
encoder = Encoder(units, BATCH_SIZE)
decoder = Decoder(embedding_dim, units, vocab_size)

###Optimizer & Loss Function

In [0]:
"""
#can use this loss function too

optimizer = tf.train.AdamOptimizer()

# We are masking the loss calculated for padding
def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)
"""

In [0]:
#this loss function requires tf.nightly otherwise it will throw an error
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_mean(loss_)

##Training

In [0]:
EPOCHS = 50

for epoch in range(EPOCHS):
    start = time.time()
    total_loss = 0
    hidden = encoder.initialize_hidden_state()
      
    for (batch, (img_tensor, target)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            features, enc_hidden = encoder(img_tensor, hidden)
            dec_hidden = enc_hidden
            dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * BATCH_SIZE, 1)
            
            for i in range(1, target.shape[1]):
                # passing the features through the decoder
                predictions, dec_hidden, _ = decoder(dec_input, features, enc_hidden)

                loss += loss_function(target[:, i], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(target[:, i], 1)
        
        total_loss += (loss / int(target.shape[1]))
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables) 
        
        optimizer.apply_gradients(zip(gradients, variables)) #tf.train.get_or_create_global_step())
        
        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, 
                                                          batch, 
                                                          loss.numpy() / int(target.shape[1])))
    print ('Epoch {} Loss {:.6f}'.format(epoch + 1, 
                                         total_loss/N_BATCH))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

W0209 10:45:30.147437 139727270086528 optimizer_v2.py:927] Gradients does not exist for variables ['encoder/batch_normalization_v2/moving_mean:0', 'encoder/batch_normalization_v2/moving_variance:0', 'encoder/batch_normalization_v2_1/moving_mean:0', 'encoder/batch_normalization_v2_1/moving_variance:0', 'encoder/batch_normalization_v2_2/moving_mean:0', 'encoder/batch_normalization_v2_2/moving_variance:0'] when minimizing the loss.


Epoch 1 Batch 0 Loss 2.4739
Epoch 1 Loss 2.473852
Time taken for 1 epoch 6.058124780654907 sec



W0209 10:45:35.757418 139727270086528 optimizer_v2.py:927] Gradients does not exist for variables ['encoder/batch_normalization_v2/moving_mean:0', 'encoder/batch_normalization_v2/moving_variance:0', 'encoder/batch_normalization_v2_1/moving_mean:0', 'encoder/batch_normalization_v2_1/moving_variance:0', 'encoder/batch_normalization_v2_2/moving_mean:0', 'encoder/batch_normalization_v2_2/moving_variance:0'] when minimizing the loss.


Epoch 2 Batch 0 Loss 2.4022
Epoch 2 Loss 2.402231
Time taken for 1 epoch 5.515782833099365 sec



W0209 10:45:41.256341 139727270086528 optimizer_v2.py:927] Gradients does not exist for variables ['encoder/batch_normalization_v2/moving_mean:0', 'encoder/batch_normalization_v2/moving_variance:0', 'encoder/batch_normalization_v2_1/moving_mean:0', 'encoder/batch_normalization_v2_1/moving_variance:0', 'encoder/batch_normalization_v2_2/moving_mean:0', 'encoder/batch_normalization_v2_2/moving_variance:0'] when minimizing the loss.


...

Epoch 48 Batch 0 Loss 1.6376
Epoch 48 Loss 1.637624
Time taken for 1 epoch 5.643473386764526 sec



W0209 10:49:55.769579 139727270086528 optimizer_v2.py:927] Gradients does not exist for variables ['encoder/batch_normalization_v2/moving_mean:0', 'encoder/batch_normalization_v2/moving_variance:0', 'encoder/batch_normalization_v2_1/moving_mean:0', 'encoder/batch_normalization_v2_1/moving_variance:0', 'encoder/batch_normalization_v2_2/moving_mean:0', 'encoder/batch_normalization_v2_2/moving_variance:0'] when minimizing the loss.


Epoch 49 Batch 0 Loss 1.6330
Epoch 49 Loss 1.633048
Time taken for 1 epoch 5.478773355484009 sec



W0209 10:50:01.317166 139727270086528 optimizer_v2.py:927] Gradients does not exist for variables ['encoder/batch_normalization_v2/moving_mean:0', 'encoder/batch_normalization_v2/moving_variance:0', 'encoder/batch_normalization_v2_1/moving_mean:0', 'encoder/batch_normalization_v2_1/moving_variance:0', 'encoder/batch_normalization_v2_2/moving_mean:0', 'encoder/batch_normalization_v2_2/moving_variance:0'] when minimizing the loss.


Epoch 50 Batch 0 Loss 1.6286
Epoch 50 Loss 1.628558
Time taken for 1 epoch 5.547411680221558 sec



##Evaluation & Prediction

In [0]:
def evaluate(image):
     
    result = ''
    image = tf.convert_to_tensor(image)
    inputs = tf.expand_dims(image, 0)
   
    hidden = decoder.reset_state(batch_size=1)
    features, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
 
    for i in range(8):
        predictions, dec_hidden, attention_weights = decoder(dec_input, features, hidden)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result = result + tokenizer.index_word[predicted_id] + ' '

        if tokenizer.index_word[predicted_id] == '<end>':
            return result

        dec_input = tf.expand_dims([predicted_id], 0)

    return result

In [0]:
evaluate(X_train[1])

'place red in a now <end> '

In [0]:
evaluate(X_train[0])

'set white with p two <end> '

In [0]:
sent

[['<start>', 'set', 'white', 'with', 'p', 'two', 'soon', '<end>'],
 ['<start>', 'place', 'red', 'in', 'a', 'zero', 'now', '<end>']]